# Fully Connected Experiments


## 1. Setup

In [ ]:
import gc
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import seaborn as sns

from collections import OrderedDict
from copy import deepcopy
from IPython import display
from random import uniform
from semitorch import (
    Viterbi, viterbi_parameters,
    Lukasiewicz, lukasiewicz_parameters,
)
from statistics import quantiles
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import FashionMNIST
from tqdm.notebook import trange

data_path = os.path.abspath("./data" if os.path.isdir("./data") else "../../data")
device = "cuda" if torch.cuda.is_available() else "cpu"


def split_dataset(dataset: torch.utils.data.Dataset):
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size

    return torch.utils.data.random_split(dataset, [train_size, test_size])

### Load iris data
[https://www.kaggle.com/datasets/uciml/iris](https://www.kaggle.com/datasets/uciml/iris)

In [ ]:
iris_df = pd.read_csv(
    os.path.join(data_path, "iris.csv"), index_col=0, dtype={"Species": "string"}
)
iris_x = torch.Tensor(iris_df.iloc[:, [0, 1, 2, 3]].to_numpy()).to(device)

iris_y = (
    iris_df["Species"]
    .map({
        "Iris-setosa": 0,
        "Iris-versicolor": 1,
        "Iris-virginica": 2,
    }).to_numpy()
)
iris_y = torch.Tensor(iris_y).to(torch.int64).to(device)
print(f"Iris dataset: input features = {iris_x.shape[1]}, "
      f"classes = {torch.unique(iris_y).shape[0]}, "
      f"samples = {len(iris_y)}")

# normalize
torch.nn.functional.normalize(iris_x, dim=0, out=iris_x)

iris_train, iris_test = split_dataset(torch.utils.data.TensorDataset(iris_x, iris_y))

iris_train_loader = DataLoader(iris_train, batch_size=8, shuffle=True)
iris_test_loader = DataLoader(iris_test, batch_size=len(iris_test), shuffle=False)

### Load heart disease data

[https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset](https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset)

In [ ]:
heart_df = pd.read_csv(os.path.join(data_path, "heart.csv"))
heart_x = (torch.Tensor(heart_df.iloc[:, range(13)].to_numpy()).to(torch.float32).to(device))
heart_y = torch.Tensor(heart_df.iloc[:, -1].to_numpy()).to(torch.int64).to(device)

print(f"Heart disease dataset: input features = {heart_x.shape[1]}, "
      f"classes = {torch.unique(heart_y).shape[0]}, "
      f"samples = {len(heart_y)}")

# normalize
torch.nn.functional.normalize(heart_x, dim=0, out=heart_x)

heart_train, heart_test = split_dataset(torch.utils.data.TensorDataset(heart_x, heart_y))

heart_train_loader = DataLoader(heart_train, batch_size=32, shuffle=True)
heart_test_loader = DataLoader(heart_test, batch_size=len(heart_test))

### Load Naitzat data

[https://github.com/topnn/topnn_framework](https://github.com/topnn/topnn_framework)

In [ ]:
circles_x, circles_y = torch.load(os.path.join(data_path, "naitzat", "circles_type_8.pt"))
circles_y = torch.squeeze(circles_y)
circles_train, circles_test = split_dataset(torch.utils.data.TensorDataset(circles_x, circles_y))
print(f"Circles dataset: input features = {circles_x.shape[1]}, "
      f"classes = {torch.unique(circles_y).shape[0]}, "
      f"samples = {len(circles_y)}")
circles_train_loader = DataLoader(circles_train, batch_size=16, shuffle=True)
circles_test_loader = DataLoader(circles_test, batch_size=len(circles_test), shuffle=True)

rings_x, rings_y = torch.load(os.path.join(data_path, "naitzat", "rings_9.pt"))
rings_y = torch.squeeze(rings_y)
rings_train, rings_test = split_dataset(torch.utils.data.TensorDataset(rings_x, rings_y))
print(f"Rings dataset: input features = {rings_x.shape[1]}, "
      f"classes = {torch.unique(rings_y).shape[0]}, "
      f"samples = {len(rings_y)}")
rings_train_loader = DataLoader(rings_train, batch_size=16, shuffle=True)
rings_test_loader = DataLoader(rings_test, batch_size=len(rings_test), shuffle=True)

spheres_x, spheres_y = torch.load(os.path.join(data_path, "naitzat", "spheres_9.pt"))
spheres_y = torch.squeeze(spheres_y)
spheres_train, spheres_test = split_dataset(torch.utils.data.TensorDataset(spheres_x, spheres_y))
print(f"Spheres dataset: input features = {spheres_x.shape[1]}, "
      f"classes = {torch.unique(spheres_y).shape[0]}, "
      f"samples = {len(spheres_y)}")
spheres_train_loader = DataLoader(spheres_train, batch_size=16, shuffle=True)
spheres_test_loader = DataLoader(spheres_test, batch_size=len(spheres_test), shuffle=True)

## FashionMNIST

In [ ]:
batch_size = 256
num_workers = 8

transforms_train = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.286,), (0.353,)),
    ]
)

transforms_test = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.286,), (0.353,)),
    ]
)

fmnist_trainset = FashionMNIST(root="../.", train=True, download=True, transform=transforms_train)
fmnist_testset = FashionMNIST(root="../.", train=False, download=True, transform=transforms_test)

fmnist_trainloader = DataLoader(fmnist_trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
fmnist_testloader = DataLoader(fmnist_testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

## 2. Models

In [ ]:
class IrisModel(nn.Module):
    def __init__(self, reverse: bool = False) -> None:
        super().__init__()

        if not reverse:
            self.layer1 = Viterbi(4, 16)
            self.layer2 = Lukasiewicz(16, 16)
            self.head = Viterbi(16, 3, bias=False)
        else:
            self.layer1 = Lukasiewicz(4, 16)
            self.layer2 = Viterbi(16, 16)
            self.head = Lukasiewicz(16, 3, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        result_1 = self.layer1(x)
        result_2 = self.layer2(result_1)

        return self.head(result_2)

In [ ]:
class HeartModel(nn.Module):
    def __init__(self, reverse: bool = False) -> None:
        super().__init__()

        if not reverse:
            self.layer1 = Viterbi(13, 16)
            self.layer2 = Lukasiewicz(16, 16)
            self.layer3 = Viterbi(16, 16)
            self.layer4 = Lukasiewicz(16, 16)
            self.head = Viterbi(16, 2, bias=False)
        else:
            self.layer1 = Lukasiewicz(13, 16)
            self.layer2 = Viterbi(16, 16)
            self.layer3 = Lukasiewicz(16, 16)
            self.layer4 = Viterbi(16, 16)
            self.head = Lukasiewicz(16, 2, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        result_1 = self.layer1(x)
        result_2 = self.layer2(result_1)
        result_3 = self.layer3(result_2)
        result_4 = self.layer4(result_3)

        return self.head(result_4)

In [ ]:
class CirclesModel(nn.Module):
    def __init__(self, reverse: bool = False) -> None:
        super().__init__()

        if not reverse:
            self.layer1 = Viterbi(2, 8)
            self.layer2 = Lukasiewicz(8, 8)
            self.layer3 = Viterbi(8, 8)
            self.layer4 = Lukasiewicz(8, 8)
            self.head = Viterbi(8, 2, bias=False)
        else:
            self.layer1 = Lukasiewicz(2, 8)
            self.layer2 = Viterbi(8, 8)
            self.layer3 = Lukasiewicz(8, 8)
            self.layer4 = Viterbi(8, 8)
            self.head = Lukasiewicz(8, 2, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        result_1 = self.layer1(x)
        result_2 = self.layer2(result_1)
        result_3 = self.layer3(result_2)
        result_4 = self.layer4(result_3)

        return self.head(result_4)

In [ ]:
class RingsAndSpheresModel(nn.Module):
    def __init__(self, reverse: bool = False) -> None:
        super().__init__()

        if not reverse:
            self.layer1 = Viterbi(3, 8)
            self.layer2 = Lukasiewicz(8, 8)
            self.layer3 = Viterbi(8, 8)
            self.layer4 = Lukasiewicz(8, 8)
            self.head = Viterbi(8, 2, bias=False)
        else:
            self.layer1 = Lukasiewicz(3, 8)
            self.layer2 = Viterbi(8, 8)
            self.layer3 = Lukasiewicz(8, 8)
            self.layer4 = Viterbi(8, 8)
            self.head = Lukasiewicz(8, 2, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        result_1 = self.layer1(x)
        result_2 = self.layer2(result_1)
        result_3 = self.layer3(result_2)
        result_4 = self.layer4(result_3)

        return self.head(result_4)

In [ ]:
class FashionModel(nn.Module):
    def __init__(self, reverse: bool = False) -> None:
        super().__init__()

        self.stem = nn.Flatten()

        if not reverse:
            self.layer_1 = Viterbi(784, 600)
            self.layer_2 = Lukasiewicz(600, 500)
            self.layer_3 = Viterbi(500 + 600, 400)
            self.layer_4 = Lukasiewicz(400 + 500, 300)
            self.layer_5 = Viterbi(300 + 400, 200)
            self.layer_6 = Lukasiewicz(200 + 300, 100)
            self.head = Viterbi(100 + 200, 10, bias=False)
        else:
            self.layer_1 = Lukasiewicz(784, 600)
            self.layer_2 = Viterbi(600, 500)
            self.layer_3 = Lukasiewicz(500 + 600, 400)
            self.layer_4 = Viterbi(400 + 500, 300)
            self.layer_5 = Lukasiewicz(300 + 400, 200)
            self.layer_6 = Viterbi(200 + 300, 100)
            self.head = Lukasiewicz(100 + 200, 10, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        result_0 = self.stem(x)

        result_1 = self.layer_1(result_0)
        result_2 = self.layer_2(result_1)
        result_3 = self.layer_3(torch.cat((result_2, result_1), dim=-1))
        result_4 = self.layer_4(torch.cat((result_3, result_2), dim=-1))
        result_5 = self.layer_5(torch.cat((result_4, result_3), dim=-1))
        result_6 = self.layer_6(torch.cat((result_5, result_4), dim=-1))

        return self.head(torch.cat((result_6, result_5), dim=-1))

In [ ]:
def count_parameters(parameters) -> int:
    return sum(p.numel() for p in parameters if p.requires_grad)

## 3. Training

In [ ]:
def accuracy(model: nn.Module, x: torch.Tensor, y: torch.Tensor) -> float:
    with torch.no_grad():
        yout = model(x)
        _, prediction = torch.max(yout.cpu(), dim=1)

        return (y.cpu() == prediction).sum().item() / float(y.numel())


def test(model: nn.Module, testloader: DataLoader) -> float:
    model.eval()
    accs = []

    with torch.no_grad():
        for x, y in testloader:
            x = x.to(device)
            accs.append(accuracy(model, x, y))

    return sum(accs) / len(accs)


def confusion_matrix(model: nn.Module, testloader: DataLoader) -> None:
    model.eval()

    conf_matrix = torch.zeros(len(fmnist_testset.classes), len(fmnist_testset.classes))

    with torch.no_grad():
        for x, y in testloader:
            x, y = x.to(device), y.to(device)
            yout = model(x)
            _, prediction = torch.max(yout.cpu(), dim=1)

            conf_matrix[y.cpu(), prediction] += 1

    plt.figure(figsize=(6, 4))

    df_cm = pd.DataFrame(conf_matrix, index=fmnist_testset.classes, columns=fmnist_testset.classes).astype(int)
    heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=15)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=15)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


def train(
        model: nn.Module,
        trainloader: DataLoader,
        testloader: DataLoader,
        optimizers: list[torch.optim.Optimizer],
        schedulers: list[torch.optim.lr_scheduler],
        loss: torch.nn.modules.loss,
        epochs: int,
        statistics_mode: bool = False  # If True: (return max acc and do not generate output)
) -> None | float:
    accs = []  # list of accuracy on the test dataset for every epoch
    trainaccs = []  # a list of the accuracies of all the training batches

    if not statistics_mode:
        fig, ax = plt.subplots(1, 1, figsize=[6, 4])
        hdisplay = display.display("", display_id=True)

    for _ in trange(epochs):
        model.train()

        for x, y in trainloader:
            x, y = x.to(device), y.to(device)
            for optimizer in optimizers:
                optimizer.zero_grad()
            yout = model(x)
            _, prediction = torch.max(yout.cpu(), dim=1)
            trainaccs.append((y.cpu() == prediction).sum().item() / float(y.numel()))

            l = loss(yout, y.squeeze())
            l.backward()
            for optimizer in optimizers:
                optimizer.step()

            for scheduler in schedulers:
                scheduler.step()

        accs.append(test(model, testloader))

        if not statistics_mode:
            ax.clear()
            ax.set_xlim(0, epochs)
            ax.set_ylim(-0.02, 1.02)
            ax.plot(
                np.linspace(0, len(accs), len(trainaccs)),
                trainaccs,
                ".",
                markersize=1.5,
                markerfacecolor=(0, 0, 1, 0.3),
            )
            ax.plot(np.linspace(1, len(accs), len(accs)), accs)
            ax.text(
                0.6 * epochs,
                0.30,
                f"max test acc = {max(accs):.2%}",
                ha="center",
                fontsize=10,
            )
            hdisplay.update(fig)

        # prevents OOM when GPU memory is tight
        torch.cuda.empty_cache()
        gc.collect()

    if not statistics_mode:
        confusion_matrix(model, testloader)

    if statistics_mode:
        return max(accs)

In [ ]:
def run_model_for_config(config: dict, best_accuracy: float) -> dict:
    loss = nn.CrossEntropyLoss()
    epochs = 20

    # Create model and set data loaders
    dataset = config["dataset"]
    if dataset == "iris":
        model = IrisModel(reverse=config["reverse"]).to(device)
        trainloader = iris_train_loader
        testloader = iris_test_loader
    elif dataset == "heart":
        model = HeartModel(reverse=config["reverse"]).to(device)
        trainloader = heart_train_loader
        testloader = heart_test_loader
    elif dataset == "circles":
        model = CirclesModel(reverse=config["reverse"]).to(device)
        trainloader = circles_train_loader
        testloader = circles_test_loader
    elif dataset in ["rings", "spheres"]:
        model = RingsAndSpheresModel(reverse=config["reverse"]).to(device)
        trainloader = rings_train_loader
        testloader = rings_test_loader
    elif dataset == "fashion":
        model = FashionModel(reverse=config["reverse"]).to(device)
        trainloader = fmnist_trainloader
        testloader = fmnist_testloader
    else:
        raise Exception(f"Unknown dataset: {dataset}")

    # Separate model parameters
    viterbi_params = viterbi_parameters(model)
    lukasiewicz_params = lukasiewicz_parameters(model)

    # Create viterbi optimizer
    viterbi_lr = config["viterbi_lr"]
    viterbi_optimizer = torch.optim.AdamW(viterbi_params, lr=viterbi_lr)
    if config["viterbi_scheduler"]:
        viterbi_scheduler = torch.optim.lr_scheduler.OneCycleLR(
            viterbi_optimizer,
            max_lr=viterbi_lr,
            anneal_strategy="linear",
            pct_start=0.3,
            three_phase=True,
            final_div_factor=1000.0,
            div_factor=10.0,
            steps_per_epoch=len(trainloader),
            epochs=epochs,
        )
    else:
        viterbi_scheduler = None

    # Create lukasiewicz optimizer
    lukasiewicz_lr = config["viterbi_lr"]
    lukasiewicz_optimizer = torch.optim.AdamW(lukasiewicz_params, lr=lukasiewicz_lr)
    if config["lukasiewicz_scheduler"]:
        lukasiewicz_scheduler = torch.optim.lr_scheduler.OneCycleLR(
            lukasiewicz_optimizer,
            max_lr=lukasiewicz_lr,
            anneal_strategy="linear",
            pct_start=0.3,
            three_phase=True,
            final_div_factor=1000.0,
            div_factor=10.0,
            steps_per_epoch=len(trainloader),
            epochs=epochs,
        )
    else:
        lukasiewicz_scheduler = None

    # Create optimizers and schedulers
    optimizers = list(filter(lambda opt: opt is not None, [viterbi_optimizer, lukasiewicz_optimizer]))
    schedulers = list(filter(lambda sch: sch is not None, [viterbi_scheduler, lukasiewicz_scheduler]))

    reported_accuracy, last_reported_accuracy = [], []
    for epoch in range(epochs):
        last_reported_accuracy = reported_accuracy

        # Feed to training function
        reported_accuracy = train_model(
            model,
            trainloader,
            testloader,
            optimizers,
            schedulers,
            loss,
        )

        # Try some stopping conditions
        if (((epoch > 2) and (reported_accuracy < 0.15))
                or ((epoch > 5) and (reported_accuracy < best_accuracy / 2))
                or ((epoch > 2) and (abs(last_reported_accuracy - reported_accuracy) < 1e-6))):
            # Run is done, no sense to train it any further
            # or bad trial
            break  # Break for-loop to report

    return {"accuracy": reported_accuracy, "config": config}


def train_model(
        model: nn.Module,
        trainloader: DataLoader,
        testloader: DataLoader,
        optimizers: list[torch.optim.Optimizer],
        schedulers: list[torch.optim.lr_scheduler],
        loss: torch.nn.modules.loss,
) -> float:
    model.train()

    for x, y in trainloader:
        x, y = x.to(device), y.to(device)
        for optimizer in optimizers:
            optimizer.zero_grad()

        l = loss(model(x), y.squeeze())
        l.backward()

        for optimizer in optimizers:
            optimizer.step()

        for scheduler in schedulers:
            scheduler.step()

        # prevents OOM when GPU memory is tight
        torch.cuda.empty_cache()
        gc.collect()

    return test(model, testloader)


def run_random_search_for_config(config: dict) -> dict:
    best_accuracy, best_config, current_config = 0, deepcopy(config), deepcopy(config)
    step_size, bad_step_counter, good_step_counter = 0.1, 0, 0
    for num_trial in range(100):
        result = run_model_for_config(current_config, best_accuracy=best_accuracy)

        if result["accuracy"] > best_accuracy:
            bad_step_counter = 0
            good_step_counter += 1

            best_accuracy, best_config = result["accuracy"], deepcopy(current_config)
        else:
            bad_step_counter += 1
            good_step_counter = 0

        if bad_step_counter > 10:
            break  # Terminate run, likely no better accuracy possible

        current_config, step_size, bad_step_counter, good_step_counter = adaptive_step_update_config(
            config=deepcopy(best_config),
            step_size=step_size,
            bad_step_counter=bad_step_counter,
            good_step_counter=good_step_counter,
        )

        if step_size < 1e-2:
            # Step size too small to make any impact, terminate run
            break

    return {"accuracy": best_accuracy, "config": best_config}


def adaptive_step_update_config(
        config: dict,
        step_size: float,
        bad_step_counter: int,
        good_step_counter: int,
) -> tuple[dict, float, int, int]:
    # Update step_size
    if bad_step_counter > 2:
        step_size /= 2
        bad_step_counter = 0
    if (good_step_counter > 2) and (step_size < 0.5):
        step_size *= 2
        good_step_counter = 0

    # Update config values
    config["viterbi_lr"] = uniform(
        config["viterbi_lr"] * (1 - step_size),
        config["viterbi_lr"] * (1 + step_size),
    )
    config["lukasiewicz_lr"] = uniform(
        config["lukasiewicz_lr"] * (1 - step_size),
        config["lukasiewicz_lr"] * (1 + step_size),
    )

    return config, step_size, bad_step_counter, good_step_counter


def find_best_model_for(config: dict) -> None:
    current_config, results = deepcopy(config), []

    for reverse in [True, False]:
        current_config["reverse"] = reverse
        for viterbi_scheduler in [True, False]:
            current_config["viterbi_scheduler"] = viterbi_scheduler
            for lukasiewicz_scheduler in [True, False]:
                current_config["lukasiewicz_scheduler"] = lukasiewicz_scheduler

                result, counter = {"accuracy": 0, "config": {}}, 0
                while counter < 5:
                    current_config = create_random_config_for_model(deepcopy(current_config))
                    print(f"Starting semiring run for config {current_config}")
                    subresult = run_random_search_for_config(current_config)
                    if subresult["accuracy"] > result["accuracy"]:
                        result = subresult
                    counter += 1

                display.clear_output(wait=True)
                results.append(result)

    sorted_results = sorted(results, key=lambda dictionary: dictionary["accuracy"], reverse=True)

    display.clear_output(wait=True)
    print(f'Best trial config with accuracy of {sorted_results[0]["accuracy"]}: {sorted_results[0]["config"]}')
    print(f"-----------------")
    print(*sorted_results, sep="\n")


def create_random_config_for_model(config: dict) -> dict:
    config["viterbi_lr"] = 10 ** uniform(-3, -1)
    config["lukasiewicz_lr"] = 10 ** uniform(-3, -1)

    return config

# Experiments

In [ ]:
find_best_model_for(config={
    "dataset": "iris",
})